In [1]:
import random
import math
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score

In [2]:
import time
start_time = time.perf_counter()

In [3]:
# TWO GENERATORS

def generate1():  
    a = random.uniform(0, 1)  
    b = random.uniform(0, 1)  
    return (a * math.cos(2 * math.pi * b), a * math.sin(2 * math.pi * b))

def generate2():  
    while True:  
        x = random.uniform(-1, 1)  
        y = random.uniform(-1, 1)  
        if x ** 2 + y ** 2 > 1:  
            continue  
        return (x, y)

In [4]:
# FULL DATA = TEST DATA

full_data = np.zeros((30, 2001), dtype=np. floating)

for i in range(15):
    for j in range(0,1999,2):
        full_data[i,j], full_data[i, j+1] = generate1()
        
for i in range(15,30):
    for j in range(0,1999,2):
        full_data[i,j], full_data[i, j+1] = generate2() 
        
full_data[0:15,2000] = 1
full_data[15:,2000] = 2

np.random.shuffle(full_data)

In [5]:
# SHORT SERIES DATA = TRAIN DATA

short_data = np.zeros((100000,3))
for i in range(50000):
    short_data[i,0],short_data[i,1] = generate1()
    short_data[i,2] = 1
    
for i in range(50000,100000):
    short_data[i,0],short_data[i,1] = generate2()
    short_data[i,2] = 2
    
np.random.shuffle(short_data)

In [6]:
# TRAIN SET

df = pd.DataFrame(short_data) 
X = df.iloc[:,:-1].values 
Y = df.iloc[:,-1:].values 
      
X_train, X_test, Y_train, Y_test = train_test_split( 
X, Y, test_size = 1/3, random_state = 0 ) 
Y_train = np.ravel(Y_train)

In [7]:
# KNN MODEL TRAINING

model_knn = KNeighborsClassifier(n_neighbors=7, weights='uniform', algorithm='auto', leaf_size=30, 
                                 p=2, metric='minkowski', metric_params=None, n_jobs=None)
model_knn.fit(X_train, Y_train)
Y_pred_knn = model_knn.predict(X_test)
print('Training accuracy: ', accuracy_score(Y_test, Y_pred_knn))

Training accuracy:  0.5797084058318833


In [8]:
# MODEL VALIDATING

X_val, Y_val = full_data[:,:-1], full_data[:,-1]
Y_pred_hint = []
for i in range(X_val.shape[0]):
    predict_line = []
    for j in range(0,700,2):
        predict_line.append(model_knn.predict(np.array([X_val[i,j],X_val[i,j+1]]).reshape(1, -1)))
    Y_pred_hint.append(round(np.mean(predict_line)))
    
print('Validating accuracy: ', accuracy_score(Y_val,Y_pred_hint))

Validating accuracy:  1.0


In [9]:
print("--- %s seconds ---" % (time.perf_counter() - start_time))

--- 5.5278148 seconds ---
